In [1]:
import baostock as bs
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import BaoStockUtil

import datetime

from RSI import DayRSI,WeekRSI,MonthRSI
from Stock import Stock
import dbutil
import KlineService

from IPython.core.debugger import set_trace

RSI_OVER_BUY = 80
RSI_OVER_SELL = 20
RSI_OVER_BUY_12 = 75
RSI_OVER_SELL_12 = 25
RSI_OVER_BUY_24 = 70
RSI_OVER_SELL_24 = 30
RSI_MIDDLE = 50

In [2]:

##
#  写RSI数据库
#
#
def writeRSIToDb(period, stockCode, stockName, rsi_df):
    dataList = []
    for index,rsi in rsi_df.iterrows():
        if period == "day":
            rsiObj = DayRSI(stockCode, stockName)
        elif period == "week":
            rsiObj = WeekRSI(stockCode, stockName)
        elif period == "month":
            rsiObj = MonthRSI(stockCode, stockName)
        elif period == "5m":
            rsiObj = FiveMinRSI(stockCode, stockName)
        elif period == "15m":
            rsiObj = FiftyMinRSI(stockCode, stockName)
        elif period == "30m":
            rsiObj = ThirtyMinRSI(stockCode, stockName)
        elif period == "60m":
            rsiObj = SixtyMinRSI(stockCode, stockName)

        rsiObj.date = rsi['date']
        rsiObj.rsi_6 = rsi['rsi_6']
        rsiObj.rsi_12 = rsi['rsi_12']
        rsiObj.rsi_24 = rsi['rsi_24']
        rsiObj.overBuy = rsi['overBuyFlag']
        rsiObj.overSell = rsi['overSellFlag']
        
        dataList.append(rsiObj.__dict__)
        
    mydb = dbutil.connectDB()
    collection = mydb[chooseRSICollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("RSI数据为空")


##
#  选择不同的Kline Collection
#
def chooseRSICollection(period):
    periodRSICollection = {
        "day" : "RSI_Day",
        "week" : "RSI_Week",
        "month" : "RSI_Month",
        "5m" : "RSI_5m",
        "15m" : "RSI_15m",
        "30m" : "RSI_30m",
        "60m" : "RSI_60m"
    }
    return periodRSICollection.get(period)


def computeRSI(klineDataFrame):
    # 剔除停盘数据
    klineDataFrame = klineDataFrame[klineDataFrame['tradeStatus'] == '1']
    rsi_12days = ta.RSI(klineDataFrame['closePrice'],timeperiod=12)
    rsi_6days = ta.RSI(klineDataFrame['closePrice'],timeperiod=6)
    rsi_24days = ta.RSI(klineDataFrame['closePrice'],timeperiod=24)
    
    rsiFrame = pd.DataFrame(klineDataFrame, columns=["date"])
    rsiFrame['rsi_6'] = rsi_6days
    rsiFrame['rsi_12'] = rsi_12days
    rsiFrame['rsi_24'] = rsi_24days
    ##添加参考线位置
    rsiFrame['overBuy'] = RSI_OVER_BUY
    rsiFrame['overSell'] = RSI_OVER_SELL
    rsiFrame['middle'] = RSI_MIDDLE

    # RSI超卖和超买
    rsi_buy_position = rsiFrame['rsi_12'] > RSI_OVER_BUY_12
    rsi_sell_position = rsiFrame['rsi_12'] < RSI_OVER_SELL_12
    rsiFrame.loc[rsi_buy_position[(rsi_buy_position == True) & (rsi_buy_position.shift() == False)].index, 'overBuyFlag'] = 'Yes'
    rsiFrame.loc[rsi_sell_position[(rsi_sell_position == True) & (rsi_sell_position.shift() == False)].index, 'overSellFlag'] = 'Yes'
    return rsiFrame

##
#  计算自起始日期起的RSI
#
#
def computeAllRSIDataOfPeriod(period, startDate):
    stockDict = KlineService.allStocks()
    endDate = str(datetime.date.today())
    processCount = 0
    failCount = 0
    for key,stock in stockDict.items():
        processCount = processCount + 1
        try:
            df = KlineService.readStockKline(key, period, startDate, endDate)
            rsi_df = computeRSI(df)
            writeRSIToDb(period, key, stock["name"], rsi_df)
        except BaseException as e:
            failCount = failCount + 1
            print ("download " + key + " error:" + str(e))
        
        if processCount % 100 == 0 and processCount > 0:
            print ("download process:", processCount, " of ", len(stockDict) ," failed:", failCount)
    return True

##
#  计算指定日期的RSI
#
#
def computeAllRSIData(period, specifiedDate):
    BaoStockUtil.customLogin()
    set_trace()
    #避免跨年问题，直接从去年开始取
    startDate = datetime.datetime.strptime(specifiedDate, "%Y-%m-%d") - datetime.timedelta(days = 365)
    
    rs = bs.query_trade_dates(start_date=datetime.datetime.strftime(startDate, "%Y-%m-%d"), end_date = datetime.date.today())
    if rs.error_code != '0':
        raise RuntimeError("交易日api调用失败了:" + rs.error_code)
    tradeDates = []
    while (rs.error_code == '0') & rs.next():
        row = rs.get_row_data()
        if row[1] == "1":
            tradeDates.append(row[0])
    if len(tradeDates) == 0:
        raise RuntimeError("取不到最新的交易日")
    set_trace()
    stockDict = KlineService.allStocks()
    processCount = 0
    failCount = 0
    startDate = None
    #每个交易日一共4小时，所以取4小时为一天，而不是24小时
    #每个计算周期一共至少需要24个节点，分钟线RSI统一除以4*60=240分钟算出所需计算数据天数，最少为一天
    #日线不用除分钟
    ## TODO 周线没想好怎么算，更别说月线了。
    if period.endswith("m"):
        daysBefore = 24 * (int(period.replace("m", "")) + 1) / (60 * 4)
    elif period == "day":
        daysBefore = 24
    else:
        endDate = date.today()
    
    print("tradeDates from ", tradeDates[len(tradeDates) - daysBefore], "to", specifiedDate)
    

In [3]:
# downloadAllKlineDataOfSingleDay("2019-09-24")

# downloadAllKlineDataOfPeriod("day", "2017-01-01")
# downloadAllStocks("2019-09-23")
# df = allStocks()

# computeAllRSIDataOfPeriod("day", "2017-01-01")

computeAllRSIData("day", "2019-09-26")

login success!
> <ipython-input-2-948200a6cd51>(111)computeAllRSIData()
    109     set_trace()
    110     #避免跨年问题，直接从去年开始取
--> 111     startDate = datetime.datetime.strptime(specifiedDate, "%Y-%m-%d") - datetime.timedelta(days = 365)
    112 
    113     rs = bs.query_trade_dates(start_date=datetime.datetime.strftime(startDate, "%Y-%m-%d"), end_date = datetime.date.today())

ipdb> n
> <ipython-input-2-948200a6cd51>(113)computeAllRSIData()
    111     startDate = datetime.datetime.strptime(specifiedDate, "%Y-%m-%d") - datetime.timedelta(days = 365)
    112 
--> 113     rs = bs.query_trade_dates(start_date=datetime.datetime.strftime(startDate, "%Y-%m-%d"), end_date = datetime.date.today())
    114     if rs.error_code != '0':
    115         raise RuntimeError("交易日api调用失败了:" + rs.error_code)

ipdb> n
> <ipython-input-2-948200a6cd51>(114)computeAllRSIData()
    112 
    113     rs = bs.query_trade_dates(start_date=datetime.datetime.strftime(startDate, "%Y-%m-%d"), end_date = datetime.da

ipdb> n
> <ipython-input-2-948200a6cd51>(117)computeAllRSIData()
    115         raise RuntimeError("交易日api调用失败了:" + rs.error_code)
    116     tradeDates = []
--> 117     while (rs.error_code == '0') & rs.next():
    118         row = rs.get_row_data()
    119         if row[1] == "1":

ipdb> c
> <ipython-input-2-948200a6cd51>(124)computeAllRSIData()
    122         raise RuntimeError("取不到最新的交易日")
    123     set_trace()
--> 124     stockDict = KlineService.allStocks()
    125     processCount = 0
    126     failCount = 0

ipdb> tradeDates
['2018-09-26', '2018-09-27', '2018-09-28', '2018-10-08', '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12', '2018-10-15', '2018-10-16', '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-22', '2018-10-23', '2018-10-24', '2018-10-25', '2018-10-26', '2018-10-29', '2018-10-30', '2018-10-31', '2018-11-01', '2018-11-02', '2018-11-05', '2018-11-06', '2018-11-07', '2018-11-08', '2018-11-09', '2018-11-12', '2018-11-13', '2018-11-14', '2018-11-15', '20

ipdb> n
> <ipython-input-2-948200a6cd51>(135)computeAllRSIData()
    133         daysBefore = 24 * (int(period.replace("m", "")) + 1) / (60 * 4)
    134     elif period == "day":
--> 135         daysBefore = 24
    136     else:
    137         endDate = date.today()

ipdb> n
> <ipython-input-2-948200a6cd51>(139)computeAllRSIData()
    136     else:
    137         endDate = date.today()
    138 
--> 139     print("tradeDates from ", tradeDates[len(tradeDates) - daysBefore], "to", specifiedDate)
    140 

ipdb> c
tradeDates from  2019-08-26 to 2019-09-26
